# 1. Imports

In [1]:
from build import Reformer

import torch

# 2. Parameters

In [2]:
FEEDBACK = True
INPUT_DIMS = 11
OUTPUT_DIMS = 4
FEEDBACK_DIMS = INPUT_DIMS+ (OUTPUT_DIMS+1 if FEEDBACK else 0)
EMBED_SIZE = 32
MAX_SEQ_LEN = 512
LAYERS = 4
HEADS = 1
KV_HEADS = None
DIFFERENTIAL = True
BIAS = False

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = torch.float32

# 3. Create model

In [4]:
MODEL = Reformer(INPUT_DIMS+FEEDBACK_DIMS, OUTPUT_DIMS, 1, EMBED_SIZE, MAX_SEQ_LEN, LAYERS, HEADS, KV_HEADS,
                 DIFFERENTIAL, 0.1, BIAS, FEEDBACK, DEVICE, DTYPE, dist='mult_var_normal')

In [5]:
MODEL.single_mode(False)

# 4. Create data

In [6]:
SEQ_LEN = 64
RECORDS = 10

In [7]:
train_inp = torch.randn(RECORDS, SEQ_LEN, INPUT_DIMS+FEEDBACK_DIMS).to(DEVICE, DTYPE)

# Test

In [8]:
torch.cuda.empty_cache()

In [9]:
with torch.no_grad():
    test_policy, test_prob = MODEL.get_action(train_inp)
MODEL.corr, MODEL.corr_indices

torch.Size([10, 64, 32]) torch.Size([10, 64, 4])


(Linear(in_features=32, out_features=6, bias=False),
 tensor([[1, 2, 2, 3, 3, 3],
         [0, 0, 1, 0, 1, 2]], device='cuda:0'))

In [10]:
test_policy[0, -4:].cpu().numpy()

array([[-0.5591284 , -0.4427582 ,  0.37089726,  0.21072271],
       [ 0.1906143 ,  0.5953509 , -1.345986  , -1.0574856 ],
       [ 0.8703771 , -1.051567  ,  1.2331686 , -0.45237875],
       [ 0.1440855 , -0.13951066, -1.4652793 ,  2.1748257 ]],
      dtype=float32)

In [11]:
test_prob[0, -4:].cpu().numpy()

array([-2.950259 , -4.781904 , -5.4954987, -7.7640896], dtype=float32)

In [42]:
with torch.no_grad():
    test_policy = MODEL(train_inp)
test_policy[0, -4:].cpu().numpy()

array([[-0.86207795, -0.31360105, -0.44722635, -0.1854907 ],
       [-0.8136764 , -0.6445205 , -0.3546932 , -0.00382823],
       [-0.79590917, -0.13938352, -0.47377133, -0.2171655 ],
       [-0.20903783, -0.0897795 , -0.67736745, -0.03809069]],
      dtype=float32)

In [12]:
with torch.no_grad():
    test_latent = MODEL.get_latent(MODEL.pol_proj, train_inp, single=True)

In [13]:
with torch.no_grad():
    test_mean = MODEL.get_mean(test_latent)

In [43]:
test_mean[0].cpu().numpy()
with torch.no_grad():
    test_std = MODEL.get_std(test_latent)

In [44]:
test_std[0].cpu().numpy()

array([[0.9219658 , 0.80742574, 0.8997422 , 0.6393007 ]], dtype=float32)

In [45]:
MODEL.mean_std

Linear(in_features=32, out_features=8, bias=False)

In [46]:
test_latent.shape

torch.Size([10, 1, 32])

In [47]:
MODEL.corr, MODEL.corr_indices

(Linear(in_features=32, out_features=6, bias=False),
 tensor([[1, 2, 2, 3, 3, 3],
         [0, 0, 1, 0, 1, 2]], device='cuda:0'))

In [48]:
del MODEL.corr

In [49]:
with torch.no_grad():
    test_dist = MODEL.dist(test_mean, test_std, test_latent)

torch.Size([10, 1, 32]) torch.Size([10, 1, 4])


In [50]:
MODEL.corr, MODEL.corr_indices

(Linear(in_features=32, out_features=6, bias=False),
 tensor([[1, 2, 2, 3, 3, 3],
         [0, 0, 1, 0, 1, 2]], device='cuda:0'))